In [1]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras import layers
import utils as ut
from typing import Dict, Optional, Tuple
from pathlib import Path
import numpy as np

In [2]:

def get_evaluate_fn(model, x_test, y_test):
    """Return an evaluation function for server-side evaluation."""

    # The `evaluate` function will be called after every round
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:

        # Update the model with the latest parameters
        model.set_weights(parameters)

        # Evaluate the model on the test data
        predictions = model.predict(x_test)
        loss = tf.keras.losses.mean_squared_error(y_test, predictions).numpy().mean().item()
        mae = tf.keras.metrics.mean_absolute_error(y_test, predictions).numpy().mean().item()

        return loss, {"mae": mae}

    return evaluate

def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Keep batch size fixed at 32, perform two rounds of training with one local epoch,
    increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 128,
        "local_epochs": 50 if server_round < 2 else 100,
    }
    return config


def evaluate_config(server_round: int):
    """Return evaluation configuration dict for each round.

    Perform five local evaluation steps on each client (i.e., use five batches) during
    rounds one to three, then increase to ten local evaluation steps.
    """
    val_steps = 5 if server_round < 4 else 10
    return {"val_steps": val_steps}

In [3]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(2,3,1)

model = tf.keras.models.Sequential(
    [
        layers.Input(shape=(8,)),  
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),  
        layers.Dense(1)  
    ]
)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)




In [4]:
# Start Flower server
history_with_noise = fl.server.start_server(
    server_address="127.0.0.1:8080",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:42:27,112 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2023-11-23 01:42:27,148 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:42:27,149 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:42:27,150 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-11-23 01:42:27,151 | server.py:91 | Evaluating initial parameters


129/129 [==============================] - 0s 820us/step


INFO flwr 2023-11-23 01:42:27,527 | server.py:94 | initial parameters (loss, other metrics): 5.977473735809326, {'mae': 2.160243272781372}
INFO flwr 2023-11-23 01:42:27,528 | server.py:104 | FL starting
DEBUG flwr 2023-11-23 01:42:59,036 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:00,441 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-23 01:43:00,443 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


129/129 [==============================] - 0s 828us/step


INFO flwr 2023-11-23 01:43:00,688 | server.py:125 | fit progress: (1, 2.9103686809539795, {'mae': 1.2954483032226562}, 33.15934220002964)
DEBUG flwr 2023-11-23 01:43:00,689 | server.py:173 | evaluate_round 1: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:01,526 | server.py:187 | evaluate_round 1 received 4 results and 0 failures
WARNING flwr 2023-11-23 01:43:01,528 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-23 01:43:01,529 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:02,885 | server.py:236 | fit_round 2 received 3 results and 0 failures


...

INFO flwr 2023-11-23 01:43:11,311 | server.py:125 | fit progress: (8, 2.0804426670074463, {'mae': 1.1065306663513184}, 43.78295259998413)
DEBUG flwr 2023-11-23 01:43:11,312 | server.py:173 | evaluate_round 8: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:12,076 | server.py:187 | evaluate_round 8 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:43:12,078 | server.py:222 | fit_round 9: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:12,399 | server.py:236 | fit_round 9 received 3 results and 0 failures


129/129 [==============================] - 0s 763us/step


INFO flwr 2023-11-23 01:43:12,630 | server.py:125 | fit progress: (9, 2.1066629886627197, {'mae': 1.1165728569030762}, 45.102012099989224)
DEBUG flwr 2023-11-23 01:43:12,632 | server.py:173 | evaluate_round 9: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:13,394 | server.py:187 | evaluate_round 9 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:43:13,396 | server.py:222 | fit_round 10: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:13,725 | server.py:236 | fit_round 10 received 3 results and 0 failures


129/129 [==============================] - 0s 758us/step


INFO flwr 2023-11-23 01:43:14,002 | server.py:125 | fit progress: (10, 2.112994909286499, {'mae': 1.1158357858657837}, 46.47409929998685)
DEBUG flwr 2023-11-23 01:43:14,003 | server.py:173 | evaluate_round 10: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:43:14,796 | server.py:187 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-11-23 01:43:14,797 | server.py:153 | FL finished in 47.26865069998894
INFO flwr 2023-11-23 01:43:14,798 | app.py:225 | app_fit: losses_distributed [(1, 2.9225618839263916), (2, 1.3900328278541565), (3, 1.4962337017059326), (4, 1.813601404428482), (5, 2.068624198436737), (6, 2.0803247690200806), (7, 2.0922287702560425), (8, 2.0831031799316406), (9, 2.1123417019844055), (10, 2.1179683804512024)]
INFO flwr 2023-11-23 01:43:14,799 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-23 01:43:14,800 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-11-23 01:43:14,800 | app.py:228 | app_fit: losses_cent

In [17]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

history_without_noise = fl.server.start_server(
    server_address="127.0.0.1:8081",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:45:43,624 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2023-11-23 01:45:43,657 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:45:43,658 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:45:43,659 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-11-23 01:45:43,659 | server.py:91 | Evaluating initial parameters


129/129 [==============================] - 0s 836us/step


INFO flwr 2023-11-23 01:45:43,913 | server.py:94 | initial parameters (loss, other metrics): 2.112994909286499, {'mae': 1.1158357858657837}
INFO flwr 2023-11-23 01:45:43,914 | server.py:104 | FL starting
DEBUG flwr 2023-11-23 01:46:16,412 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:17,938 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-23 01:46:17,940 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


129/129 [==============================] - 0s 817us/step


...

INFO flwr 2023-11-23 01:46:29,107 | server.py:125 | fit progress: (8, 2.1554107666015625, {'mae': 1.121968150138855}, 45.192388500028756)
DEBUG flwr 2023-11-23 01:46:29,108 | server.py:173 | evaluate_round 8: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:29,849 | server.py:187 | evaluate_round 8 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:46:29,851 | server.py:222 | fit_round 9: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:30,222 | server.py:236 | fit_round 9 received 3 results and 0 failures


129/129 [==============================] - 0s 992us/step


INFO flwr 2023-11-23 01:46:30,500 | server.py:125 | fit progress: (9, 2.1584832668304443, {'mae': 1.1214284896850586}, 46.58567870000843)
DEBUG flwr 2023-11-23 01:46:30,501 | server.py:173 | evaluate_round 9: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:31,436 | server.py:187 | evaluate_round 9 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:46:31,437 | server.py:222 | fit_round 10: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:31,965 | server.py:236 | fit_round 10 received 3 results and 0 failures


129/129 [==============================] - 0s 875us/step


INFO flwr 2023-11-23 01:46:32,219 | server.py:125 | fit progress: (10, 2.159140110015869, {'mae': 1.1267071962356567}, 48.304939199995715)
DEBUG flwr 2023-11-23 01:46:32,220 | server.py:173 | evaluate_round 10: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:46:33,087 | server.py:187 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-11-23 01:46:33,088 | server.py:153 | FL finished in 49.173792899993714
INFO flwr 2023-11-23 01:46:33,090 | app.py:225 | app_fit: losses_distributed [(1, 2.0978851318359375), (2, 2.1095409393310547), (3, 2.103264808654785), (4, 2.119640588760376), (5, 2.1284210681915283), (6, 2.1420881748199463), (7, 2.1244852542877197), (8, 2.13070011138916), (9, 2.12174916267395), (10, 2.157604455947876)]
INFO flwr 2023-11-23 01:46:33,090 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-23 01:46:33,090 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-11-23 01:46:33,091 | app.py:228 | app_fit: losses_central

In [18]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

history_with_minus_noise = fl.server.start_server(
    server_address="127.0.0.1:8082",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:48:40,584 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2023-11-23 01:48:40,609 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:48:40,611 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:48:40,611 | server.py:276 | Requesting initial parameters from one random client


In [ ]:
print(history)

History (loss, distributed):
	round 1: 2.647073984146118
	round 2: 1.4041863679885864
	round 3: 1.6445988416671753
	round 4: 1.935247778892517
	round 5: 2.0753211975097656
	round 6: 2.0844905376434326
	round 7: 2.086702585220337
	round 8: 2.0768063068389893
	round 9: 2.1068055629730225
	round 10: 2.0973358154296875
History (loss, centralized):
	round 0: 5.924674987792969
	round 1: 2.647073984146118
	round 2: 1.4041863679885864
	round 3: 1.6445988416671753
	round 4: 1.935247778892517
	round 5: 2.0753211975097656
	round 6: 2.0844905376434326
	round 7: 2.086702585220337
	round 8: 2.0768063068389893
	round 9: 2.1068055629730225
	round 10: 2.0973358154296875
History (metrics, centralized):
{'mae': [(0, 2.1480019092559814), (1, 1.211744785308838), (2, 0.9283948540687561), (3, 1.0063337087631226), (4, 1.0710625648498535), (5, 1.1068549156188965), (6, 1.107938289642334), (7, 1.1093168258666992), (8, 1.1095515489578247), (9, 1.1134757995605469), (10, 1.115409255027771)]}
